# LLM Model for Sentiment Analysis - T5 (Text-to-Text Transfer Transformer)

# Library Imports

In [1]:
# pip install transformers
# pip install datasets
# pip install sentencepiece
# pip install evaluate

import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    T5ForSequenceClassification, # T5 for specifically classification task
    TrainingArguments,
    Trainer
)
import evaluate
import torch
from torch.utils.data import DataLoader

/opt/anaconda3/envs/temp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data Loading and Train-Test-Split

In [2]:
df = pd.read_csv("DisneylandReviews.csv",
                 encoding = 'latin-1')
df.head()

,Review_ID,Rating,Year_Month,Reviewer_Location,Review_Text,Branch
0,670772142,4,2019-4,Australia,If you've ever been to Disneyland anywhere you...,Disneyland_HongKong
1,670682799,4,2019-5,Philippines,Its been a while since d last time we visit HK...,Disneyland_HongKong
2,670623270,4,2019-4,United Arab Emirates,Thanks God it wasn t too hot or too humid wh...,Disneyland_HongKong
3,670607911,4,2019-4,Australia,HK Disneyland is a great compact park. Unfortu...,Disneyland_HongKong
4,670607296,4,2019-4,United Kingdom,"the location is not in the city, took around 1...",Disneyland_HongKong


In [3]:
# keep the text and label
df = dict(df[['Review_Text', 'Rating']])

In [4]:
# convert to Hugging Face dataset
dataset = Dataset.from_dict(df)
dataset

Dataset({
    features: ['Review_Text', 'Rating'],
    num_rows: 42656
})

# Preprocessing

In [5]:
model_name = 'google-t5/t5-small'

In [6]:
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
def preprocess_function(reviews):

    # task
    inputs = [f"classify sentiment: {review}" for review in reviews['Review_Text']]

    # tokenize inputs
    model_inputs = tokenizer(inputs,
                             padding = True,
                             truncation = True,
                             max_length = 128)
    
    # append labels
    model_inputs['labels'] = reviews['Rating']

    return model_inputs

# apply preprocessing
tokenized_datasets = dataset.map(preprocess_function,
                                 batched = True)

Map: 100%|██████████| 42656/42656 [00:09<00:00, 4583.56 examples/s]


In [8]:
# train test split
tokenized_datasets = tokenized_datasets.train_test_split(test_size = 0.2)
train_dataset = tokenized_datasets['train']
eval_dataset = tokenized_datasets['test']

# T5 (Text-to-Text Transfer Transformer) Model

In [9]:
t5_model = T5ForSequenceClassification.from_pretrained(model_name, 
                                                      num_labels = 5)

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google-t5/t5-small and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Training Arguments

In [ ]:
training_args = TrainingArguments(
    output_dir = "./results",
    eval_strategy = "steps",
    eval_steps = 500,
    save_steps = 1000,
    save_total_limit = 2,
    logging_steps = 100,
    learning_rate = 1e-4,
    per_device_train_batch_size = 8, # small batch to fit GPU
    per_device_eval_batch_size = 8,
    gradient_accumulation_steps = 4, # effective batch size = train batch size x gradient accumulation steps (8 x 4 = 32)
    num_train_epochs = 1,
    weight_decay = 0.01,
    report_to = 'none' # avoids unnecessary logging
 )

## Trainer

In [18]:
trainer = Trainer(
    model = t5_model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    tokenizer = tokenizer
)


/var/folders/x8/_2v65kgx1zxc0k4y6d9hts5h0000gn/T/ipykernel_11132/709607782.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


## Model Train

In [19]:
%%time
trainer.train()

Step,Training Loss,Validation Loss


CPU times: user 20.5 s, sys: 10.1 s, total: 30.6 s
Wall time: 28.6 s


KeyboardInterrupt: 

## Model Inference

In [ ]:
test_sentence = "I hate this amusement park."

# tokenize input
inputs = tokenizer("Classify sentiment: " + test_sentence, 
                   return_tensors="pt")

# produce output 
outputs = t5_model(**inputs)

# get predicted label (highest probability)
pred_label = outputs.logits.argmax(dim = -1).item()

print("Predicted label:", pred_label)